In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression
import pydot
from IPython.display import Image
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("BANKNIFTYFUTALL.csv")
feature_cols_JUN = ['trade', 'event', 'ob', 'spreadreturn','niftyidevent', 'niftyidob','niftyidindex']

#data.dropna(subset=['return800'], axis=0)

data.drop([data.index[457897]],inplace=True)
#data.isnull().sum(axis = 0)


In [7]:
X= data[feature_cols_JUN]
Y = data.return800
print(X)
print(Y)

           trade     event        ob  spreadreturn  niftyidevent  niftyidob  \
0       1.887790 -0.480131  0.691948      0.340223      0.559968  -1.553040   
1       1.887790 -0.416937  1.592650     -0.125436      0.537654  -1.530060   
2       2.502120  0.159981  0.725512      0.350363      0.537654  -1.605290   
3       1.935330  0.274211  1.238070     -0.278274      1.031850  -0.012095   
4       1.759630  0.847955  0.864624      0.287151      0.733237   1.780540   
5       1.759630  0.707320  0.430883     -1.020880     -0.720371   0.510119   
6       1.759630  0.625122  0.137820      0.334935      0.938224   1.195730   
7       1.970140  1.519540  1.051380      0.339038      0.739260  -0.313306   
8       1.970140  1.464540  0.948582      0.321138      0.278489  -1.853770   
9       1.970140  1.372050  0.608076      1.129120      0.408007  -1.216400   
10      1.469680  1.456080 -0.992390     -1.415430      0.408007  -1.216400   
11      1.469680  1.114180 -1.069940     -0.532296  

##### Applying Linear Regression by dividing the dataset into two parts

In [4]:
from sklearn.linear_model import LinearRegression
Index_label1 = X[X['ob']<-0.527].index.tolist() 
x_model_1 = X.iloc[Index_label1]
y_model_1=Y.iloc[Index_label1]

Index_label2 = X[X['ob']>=-0.527].index.tolist()
x_model_2=X.iloc[Index_label2]
y_model_2 = Y.iloc[Index_label2]

reg1= LinearRegression()
reg1.fit(x_model_1,y_model_1)
reg2= LinearRegression()
reg2.fit(x_model_2,y_model_2)
reg= LinearRegression()
reg.fit(X,Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

##### Comparing the scores of the model Before and After split
- It can be observed that score of model after splitting has increased drastically

In [6]:
r_sq1 = reg1.score(x_model_1,y_model_1)
print("r_sq1 ",r_sq1)
r_sq2 = reg2.score(x_model_2,y_model_2)
print("r_sq2 ",r_sq2)
r_sq = reg.score(X,Y)
print("r_sq ",r_sq)

predict1 = reg1.predict(x_model_1)
predict2 = reg2.predict(x_model_2)
pred = reg.predict(X)

pred= pd.Series(data=pred)
predict1 = pd.Series(data=predict1)
predict2 = pd.Series(data=predict2)

correlation1 = y_model_1.corr(predict1)
correlation2 = y_model_2.corr(predict2)
corre= Y.corr(pred)
print("correlation_after_split :",correlation1)
print("correlation_after_spit :",correlation2)
print("corr_before_split:",corre)

r_sq1  0.08174825258539253
r_sq2  0.011964269512279468
r_sq  0.0031935721278721907
correlation_after_split : -0.000617111161783011
correlation_after_spit : -0.0001303789433374712
corr_before_split: 0.05651169903544089


##### Calculating the correlation between predicted and actual values manually

In [7]:
l1=[]
l2=[]
for i in range(len(Y)):
    if(Y[i]>0):
        l1.append(1)
    elif(Y[i]<0):
        l1.append(-1)
    else:
        l1.append(0)
        
for i in range(len(pred)):
    if(pred[i]>0):
        l2.append(1)
    elif(pred[i]<0):
        l2.append(-1)
    else:
        l2.append(0)

print(len(l1))
print(len(l2))
pred= pd.Series(data=pred)
pred1 = pd.Series(data=l1)
pred2 = pd.Series(data=l2)
corr1 = y_model_1.corr(pred1)
corr2 = y_model_2.corr(pred2)
co= pred1.corr(pred2)
print(co)

457897
457897
0.06055169105064174


##### Applying Decision Tree Regression for our dataset
- Decision trees regression normally use mean squared error (MSE) to decide to split a node in two or more sub-nodes
- Here, many different values of test split and train split are taken to check if the model is overfitting or not
- Overfitting occurs when on changing the size of dataset and it parameters the score of train set increases whereas the score of test set decreases.

In [8]:
x_train,x_test,y_train,y_test= train_test_split(X,Y,test_size=0.3,random_state=1)

dt = DecisionTreeRegressor(max_depth=4,min_samples_split=2)
dt.fit(x_train,y_train)

from sklearn.metrics import r2_score,mean_squared_error
y_pred = dt.predict(x_test)
mse=mean_squared_error(y_test,y_pred)
rmse= np.sqrt(mse)
print("rmse",rmse)

score_train=dt.score(x_train,y_train)
print("score for train",score_train)
print("score fro test",dt.score(x_test,y_test))

rmse 4.01689595180483
score for train 0.08252301423604103
score fro test 0.08137200305545478


##### Below are some reults with different parameters
##for depth=1, split at 0.2,0.3,0.4,0.5
- rmse 4.145847605849474
- score for train 0.02226440625077197
- score fro test 0.02283569743557245

- rmse 4.144504581414699
- score for train 0.022507148317065573
- score fro test 0.022079027012244956

- rmse 4.144577992334352
- score for train 0.022997720054378945
- score fro test 0.021442723908110643

- rmse 4.148930982547038
- score for train 0.02281081351563241
- score fro test 0.021948244176629328

##for depth=2 spit at 0.2,0.3,0.4,0.5
- rmse 4.073192484629125
- score for train 0.0570762952822117
- score fro test 0.05678479664169356

- rmse 4.07156993702865
- score for train 0.05735271793947039
- score fro test 0.05619492063668153

- rmse 4.071692793025529
- score for train 0.057895660733723764
- score fro test 0.05555727862956494

- rmse 4.074746247753332
- score for train 0.05716523872980195
- score fro test 0.056611553379002255

##for depth=3, split at 0.2,0.3,0.4
- rmse 4.0468492820746285
- score for train 0.06929428369582413
- score fro test 0.06894575381720713

- rmse 4.046364697810738
- score for train 0.06967525494007609
- score fro test 0.06784408802936615

- rmse 4.046033401840228
- score for train 0.07049563186003682
- score fro test 0.06742333381827836

##for depth=4
- rmse 4.017543554643882
- score for train 0.08223427935788684
- score fro test 0.08238160255808935

- rmse 4.01689595180483
- score for train 0.08252301423604103
- score fro test 0.08137200305545478

- rmse 4.0185934043951566
- score for train 0.083427696916616
- score fro test 0.08002981729724767




##### Applying DecisionTreeRegressor on the data by changing the value of parameters like max_depth and min_samples_split
- Minimum samples for a node split: Defines the minimum number of samples (or observations) which are required in a node to be considered for splitting.

- Minimum samples for a terminal node (leaf): Defines the minimum samples (or observations) required in a terminal node or leaf.

- Maximum depth of the tree (vertical depth): Used to control over-fitting as higher depth will allow the model to learn relations very specific to a particular sample


In [5]:
x_train,x_test,y_train,y_test= train_test_split(X,Y,test_size=0.4,random_state=1)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((274738, 7), (183159, 7), (274738,), (183159,))

In [6]:
dt = DecisionTreeRegressor(max_depth=1,min_samples_split=2)
dt.fit(x_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=1, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [10]:
dt.predict([[1.970140, 1.519540,  1.051380,0.339038   , 0.739260,  -0.313306  , 0.00000]])

array([0.31432045])

In [11]:
from sklearn.metrics import r2_score,mean_squared_error
y_pred = dt.predict(x_test)
mse=mean_squared_error(y_test,y_pred)
rmse= np.sqrt(mse)
rmse

4.144577992334352

In [12]:
r2_score(y_test,y_pred)

0.02144272390811064

In [13]:
score_train=dt.score(x_train,y_train)
score_train

0.022997720054378945

In [14]:
score=dt.score(x_test,y_test)
score

0.021442723908110643

In [15]:
dt2 = DecisionTreeRegressor(max_depth=2,min_samples_split=2)
dt2.fit(x_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [16]:
from sklearn.metrics import r2_score,mean_squared_error
y_pred = dt2.predict(x_test)
mse=mean_squared_error(y_test,y_pred)
rmse= np.sqrt(mse)
print("rmse",rmse)

score_train=dt2.score(x_train,y_train)
print("score for train",score_train)
print("score fro test",dt2.score(x_test,y_test))

rmse 4.071692793025529
score for train 0.05789566073372354
score fro test 0.05555727862956494


In [25]:
dt3 = DecisionTreeRegressor(max_depth=3,min_samples_split=2)
dt3.fit(x_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=3, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [26]:
from sklearn.metrics import r2_score,mean_squared_error
y_pred = dt3.predict(x_test)
mse=mean_squared_error(y_test,y_pred)
rmse= np.sqrt(mse)
print("rmse",rmse)

score_train=dt3.score(x_train,y_train)
print("score for train",score_train)
print("score fro test",dt3.score(x_test,y_test))

rmse 4.046033401840228
score for train 0.07049563186003682
score fro test 0.06742333381827825


In [19]:
dt4 = DecisionTreeRegressor(max_depth=4,min_samples_split=2)
dt4.fit(x_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [20]:
from sklearn.metrics import r2_score,mean_squared_error
y_pred = dt4.predict(x_test)
mse=mean_squared_error(y_test,y_pred)
rmse= np.sqrt(mse)
print("rmse",rmse)

score_train=dt4.score(x_train,y_train)
print("score for train",score_train)
print("score fro test",dt4.score(x_test,y_test))

rmse 4.0185934043951566
score for train 0.083427696916616
score fro test 0.08002981729724767


In [21]:
dt5 = DecisionTreeRegressor(max_depth=5,min_samples_split=2)
dt5.fit(x_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [22]:
from sklearn.metrics import r2_score,mean_squared_error
y_pred = dt5.predict(x_test)
mse=mean_squared_error(y_test,y_pred)
rmse= np.sqrt(mse)
print("rmse",rmse)

score_train=dt5.score(x_train,y_train)
print("score for train",score_train)
print("score for test",dt5.score(x_test,y_test))

rmse 4.004325911148841
score for train 0.09143252653695733
score for test 0.08655068993141246


In [23]:
dt6 = DecisionTreeRegressor(max_depth=6,min_samples_split=3)
dt6.fit(x_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=6, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=3, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [58]:
from sklearn.metrics import r2_score,mean_squared_error
y_pred = dt6.predict(x_test)
mse=mean_squared_error(y_test,y_pred)
rmse= np.sqrt(mse)
print("rmse",rmse)

score_train=dt6.score(x_train,y_train)
print("score for train",score_train)
print("score for test",dt6.score(x_test,y_test))

rmse 3.991361685485709
score for train 0.09704695337027669
score for test 0.09301379765271489


##### Constructing the tree

In [8]:
from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz  
dot_data = StringIO()
export_graphviz(dt, out_file='dot_data_rec.dot',  
                         feature_names=['trade', 'event', 'ob', 'spreadreturn','niftyidevent', 'niftyidob','niftyidindex'],  
                         class_names='return800',  
                         filled=True, rounded=True,  
                         special_characters=True)  
#graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
#Image(graph.create_png())

In [34]:
#at maxdepth=1, got score =0.022393481563377282
#at maxdepth=2, got score= 0.05670479000003592
#at maxdepth=3, got score= 0.0690640327405383
#at maxdepth=4, got score= 0.08066786342007692
#when max_depth value not specified, got score= 0.9555801579283528(overfitting)
score=dt3.score(X,Y)
score

0.06922466540674233

In [32]:
regr = BaggingRegressor(base_estimator=LinearRegression(),n_estimators=10, random_state=0).fit(X, Y)

In [33]:
regr.score(X,Y)

0.003192241424595621